# DBSCAN

Este algoritmo funciona en base a la distribución dimensional de los puntos de entrada, como el resto de métodos que hemos visto hasta ahora. En este caso, el algoritmo agrupa los puntos de entrada dependiendo de la densidad espacial de los mismos.

En concreto, los pasos que sigue el algoritmo son los siguientes:

1. Para cada punto del conjunto de datos de entrada, calcula quiénes son sus vecinos en un radio de similitud menor a $\varepsilon$. Además, marca como puntos núcleo aquellos que tienen al menos $minPoints$ dentro de su radio.
2. Busca las componentes conectadas que forman los puntos núcleo dentro del radio $\varepsilon$. Éstos serán los clusters.
3. Para cada punto no núcleo, se asigna a un clúster ya existente si se encuentra dentro del radio $\varepsilon$. Si no hay ninguno, se considera al punto como ruido o _outlier_.

Por tanto tenemos tres hiper-parámetros:

- $\varepsilon$ (`eps`) para determinar el radio de similitud
- $minPoints$ para determinar cuántos puntos en su vecindad hacen a otro puntos convertirse en núcleo
- la función de similitud de los puntos

La siguiente imagen, [extraída de Wikpedia](https://es.wikipedia.org/wiki/DBSCAN), ilustra el proceso. Los puntos marcados como A son puntos núcleo. Los puntos B y C son densamente alcanzables desde A y densamente conectados con A, y pertenecen al mismo clúster. El punto N es un punto ruidoso que no es núcleo ni densamente alcanzable.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/DBSCAN-Illustration.svg/1920px-DBSCAN-Illustration.svg.png" width=600>


Vamos a probar el algoritmo usando un dataset del repositorio [UCI](https://archive.ics.uci.edu/ml/datasets/Online+Shoppers+Purchasing+Intention+Dataset). Es un conjunto de datos de características sobre las sesiones online de compradores en una tienda virtual.

Según la propia descripción del conjunto de datos:



> The dataset consists of 10 numerical and 8 categorical attributes.
The 'Revenue' attribute can be used as the class label.
>
> "Administrative", "Administrative Duration", "Informational", "Informational Duration", "Product Related" and "Product Related Duration" represent the number of different types of pages visited by the visitor in that session and total time spent in each of these page categories. The values of these features are derived from the URL information of the pages visited by the user and updated in real time when a user takes an action, e.g. moving from one page to another.
>
> The "Bounce Rate", "Exit Rate" and "Page Value" features represent the metrics measured by "Google Analytics" for each page in the e-commerce site. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. The value of "Exit Rate" feature for a specific web page is calculated as for all pageviews to the page, the percentage that were the last in the session. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction. The "Special Day" feature indicates the closeness of the site visiting time to a specific special day (e.g. Mother’s Day, Valentine's Day) in which the sessions are more likely to be finalized with transaction. The value of this attribute is determined by considering the dynamics of e-commerce such as the duration between the order date and delivery date.
>
>The dataset also includes operating system, browser, region, traffic type, visitor type as returning or new visitor, a Boolean value indicating whether the date of the visit is weekend, and month of the year.

Por tanto, tenemos 7 variables categóricas que habrá que transformar.


In [ ]:
import pandas as pd

raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv')
target = raw['Revenue']
raw.drop('Revenue', axis=1, inplace=True)
raw.head(5)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

nominal_features = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']
numeric_features = list(set(raw.columns) - set(nominal_features))
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(copy=False), numeric_features),
        ('cat', OneHotEncoder(categories='auto'), nominal_features)])

datos = preprocessor.fit_transform(raw)

In [ ]:
datos.shape

Puesto que el hiper-parámetro $\varepsilon$ se basa en la distancia, para ajustarlo, es interesante conocer la distancía máxima existente:

In [ ]:
from sklearn.neighbors import kneighbors_graph
distancias = kneighbors_graph(datos, 149, mode='distance')

In [ ]:
distancias.todense().flatten().max()

Ahora, podemos calcular medidas de calidad para DBSCAN:

In [ ]:
from sklearn.cluster import DBSCAN

modelo_dbs = DBSCAN(eps=7.0, min_samples=75, metric='manhattan')

In [ ]:
labs = modelo_dbs.fit_predict(datos)

In [ ]:
from sklearn.metrics import silhouette_score, adjusted_rand_score

print('Silhouette: ', silhouette_score(datos, labs))
print('Rand Index (con ground truth): ', adjusted_rand_score(target, labs))

Hay un proceso para determinar valores óptimos (locales) para los hyper-parámetros del algoritmo:

1. Se calculan las distancias desde cada punto a los $minPoints-1$ más cercanos
2. Se ordenan las distancias, se plotean y se elige el $\varepsilon$ donde la curva es más pronunciada


In [ ]:
import numpy as np

from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt

neigh = NearestNeighbors(n_neighbors=75)
nbrs = neigh.fit(datos)
distances, indices = nbrs.kneighbors(datos)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
modelo_dbs = DBSCAN(eps=0.7, min_samples=75)
labs = modelo_dbs.fit_predict(datos)
print('Silhouette: ', silhouette_score(datos, labs))
print('Rand Index (con ground truth): ', adjusted_rand_score(target, labs))

## Ejercicio
Realiza una optimización grid para diferentes valores de $minPoints$ así como de métricas de similitud para ver cómo afecta al rendimiento del modelo DBSCAN y sus resultados en Silhouette y Rand Index.

<hr>

Creado por **Raúl Lara** (raul.lara@upm.es) y **Fernando Ortega** (fernando.ortega@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">